# Dichotic Pitch Analysis

This notebook analyzes and visualizes dichotic pitch waveforms and their spectra.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from dichotic_pitch import dichotic_pitch
from scipy import signal
from IPython.display import Audio

# Set up plotting style
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = [12, 8]

In [ ]:
# Generate dichotic pitch signal
sbr = 1.0  # signal-to-background ratio
peak = 220  # frequency in Hz
width = 20  # bandwidth in Hz
ts_sig = -0.7  # signal time shift in ms
ts_back = 0.0  # background time shift in ms
fs = 16000  # sampling rate
duration = 2  # duration in seconds

# Generate the signal
dp, fs = dichotic_pitch(
    sbr=sbr,
    peak=peak,
    width=width,
    ts_sig=ts_sig,
    ts_back=ts_back,
    samp_sec=fs,
    duration=duration
)
Audio(data=dp.T, rate=fs)

In [ ]:
# Binaural beats
framerate = 44100
t = np.linspace(0,5,framerate*5)
dataleft = np.sin(2*np.pi*220*t)
dataright = np.sin(2*np.pi*229*t)
#Audio([dataleft, dataright],rate=framerate)
Audio(np.vstack([dataleft, dataright]),rate=framerate)

## Time Domain Analysis

In [ ]:
# Plot time series
time = np.arange(len(dp)) / fs

fig, (ax1, ax2) = plt.subplots(2, 1, height_ratios=[1, 1])
fig.suptitle('Dichotic Pitch Waveforms')

# Plot first 1000 samples for better visibility
samples_to_plot = 1000

# Left channel
ax1.plot(time[:samples_to_plot] * 1000, dp[:samples_to_plot, 0], label='Left Channel')
ax1.set_ylabel('Amplitude')
ax1.set_title('Left Channel')
ax1.grid(True)

# Right channel
ax2.plot(time[:samples_to_plot] * 1000, dp[:samples_to_plot, 1], label='Right Channel', color='orange')
ax2.set_xlabel('Time (ms)')
ax2.set_ylabel('Amplitude')
ax2.set_title('Right Channel')
ax2.grid(True)

plt.tight_layout()
plt.show()

## Frequency Domain Analysis

In [ ]:
def plot_spectrum(signal_data, fs, ax, title):
    """Plot the power spectrum of a signal"""
    # Note: signal.welch instead of signal.welch
    f, Pxx = signal.welch(signal_data, fs, nperseg=4096)
    ax.semilogy(f, Pxx)
    ax.set_xlabel('Frequency (Hz)')
    ax.set_ylabel('Power Spectral Density')
    ax.set_title(title)
    ax.grid(True)
    ax.set_xlim(0, 2000)  # Focus on frequencies up to 2kHz

# Create figure for spectra
fig, (ax1, ax2) = plt.subplots(2, 1, height_ratios=[1, 1])
fig.suptitle('Power Spectra of Dichotic Pitch Signals')

# Plot spectra
plot_spectrum(dp[:, 0], fs, ax1, 'Left Channel Spectrum')
plot_spectrum(dp[:, 1], fs, ax2, 'Right Channel Spectrum')

# Add vertical lines for the peak frequency and bandwidth
for ax in [ax1, ax2]:
    ax.axvline(x=peak, color='r', linestyle='--', alpha=0.5, label='Peak Frequency')
    ax.axvline(x=peak-width/2, color='g', linestyle=':', alpha=0.5, label='Bandwidth')
    ax.axvline(x=peak+width/2, color='g', linestyle=':', alpha=0.5)
    ax.legend()

plt.tight_layout()
plt.show()

## Cross-correlation Analysis

In [ ]:
# Calculate and plot cross-correlation between channels
max_lag = int(0.01 * fs)  # 10ms maximum lag
lags = np.arange(-max_lag, max_lag + 1) / fs * 1000  # Convert to ms
xcorr = signal.correlate(dp[:, 0], dp[:, 1], mode='same')
xcorr = xcorr[len(xcorr)//2-max_lag:len(xcorr)//2+max_lag+1]

plt.figure(figsize=(12, 4))
plt.plot(lags, xcorr)
plt.xlabel('Lag (ms)')
plt.ylabel('Cross-correlation')
plt.title('Cross-correlation between Left and Right Channels')
plt.grid(True)

# Add vertical lines for the time shifts
plt.axvline(x=-ts_sig, color='r', linestyle='--', alpha=0.5, label='Signal Time Shift')
plt.axvline(x=-ts_back, color='g', linestyle='--', alpha=0.5, label='Background Time Shift')
plt.legend()

plt.tight_layout()
plt.show()

In [14]:
def multi_bandpass(n, low_freqs, high_freqs):
    """Create multiple bandpass filters"""
    # Create frequency array
    freqs = np.fft.rfftfreq(n)
    filt = np.zeros(n//2 + 1)
    
    # Convert input to arrays if they're not already
    low_freqs = np.atleast_1d(low_freqs)
    high_freqs = np.atleast_1d(high_freqs)
    
    # Create rectangular bandpass filter
    for low, high in zip(low_freqs, high_freqs):
        filt += np.where((abs(freqs) >= low) & (abs(freqs) <= high), 1, 0)
    
    return filt

In [ ]:
sbr = 3
np.max([1 - sbr - 1, -1])

In [ ]:
samp_sec = 8000
duration = 0.25
n = round(duration * samp_sec)
f_nyquist = samp_sec / 2
width = 100
peak = 440
low, high = (peak-width / 2) / f_nyquist, (peak+width / 2) / f_nyquist
sig_filter = multi_bandpass(n, low, high)
back_filter = sig_filter.copy()
renorm_filter = sig_filter.copy()

# Scale filters
sig_filter = sig_filter * sbr
back_filter = back_filter * np.max([1 - sbr - 1, -1]) + 1
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 3))
ax1.plot(sig_filter)
ax2.plot(back_filter)
plt.tight_layout()
plt.show()


In [ ]:
amp = np.sqrt(n)

# Create full phase arrays
sig_phase = np.concatenate([[0], np.random.rand(n//2) * 2 * np.pi - np.pi])
back_phase = np.concatenate([[0], np.random.rand(n//2) * 2 * np.pi - np.pi])
# Combine amplitude and phase
sig = amp * np.exp(1j * sig_phase)
back = amp * np.exp(1j * back_phase)
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 3))
ax1.plot(sig)
ax2.plot(back)
plt.tight_layout()
plt.show()

In [ ]:
sig = np.real(np.fft.irfft(sig_filter * sig, n))
back = np.real(np.fft.irfft(back_filter * back, n))
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 3))
ax1.plot(sig)
ax2.plot(back)
plt.tight_layout()
plt.show()

In [ ]:
np.fft.rfftfreq(20)

In [ ]:
freqs = np.fft.rfftfreq(20)
np.abs(freqs)

In [ ]:
x = np.linspace(-1, 1, 20)
print(x)
xf = np.fft.rfft(x)
print(xf)
np.fft.irfft(xf, len(x))

In [73]:
sbr = 1.0
peak = 440
width = 50
ts_sig = -0.7 
ts_back = 0.7 
samp_sec = 8192
duration = 1.0

peak = np.atleast_1d(peak)
width = np.atleast_1d(width)
f_nyquist = samp_sec / 2

# Calculate number of samples
n = round(duration * samp_sec)
n += n % 2  # ensure n is even

# Create pseudo-noise
amp = np.sqrt(n)

# Create full phase arrays
sig_phase = np.concatenate([[0], np.random.rand(n//2) * 2 * np.pi - np.pi])
back_phase = np.concatenate([[0], np.random.rand(n//2) * 2 * np.pi - np.pi])

# Combine amplitude and phase
sig = amp * np.exp(1j * sig_phase)
back = amp * np.exp(1j * back_phase)

# Create filters
sig_filter = multi_bandpass(n, (peak-width / 2) / f_nyquist, (peak+width / 2) / f_nyquist)
back_filter = sig_filter.copy()
renorm_filter = sig_filter.copy()

# Scale filters
sig_filter = sig_filter * sbr
back_filter = back_filter * np.maximum((1-sbr-1), -1) + 1

# Renormalization for sbr < 1
if sbr < 1:
    renorm = 1/np.sqrt(sbr**2 + (1-sbr)**2)
    renorm_filter = renorm_filter * (renorm - 1) + 1
else:
    renorm_filter = None

# Transform to time domain
sig = np.real(np.fft.irfft(sig_filter * sig, n))
back = np.real(np.fft.irfft(back_filter * back, n))

# Create stereo channels
dp = np.zeros((n, 2))
dp[:, 0] = sig + back

# Apply time shifts
sig = shift(sig, round(ts_sig/1000 * samp_sec))
back = shift(back, round(ts_back/1000 * samp_sec))
dp[:, 1] = sig + back

# Apply renormalization if necessary
if renorm_filter is not None:
    for ch in range(2):
        ft = np.fft.fftshift(np.fft.fft(dp[:, ch]))
        dp[:, ch] = np.real(np.fft.ifft(np.fft.ifftshift(ft * renorm_filter)))

# Normalize to prevent clipping
dp = dp / np.max(np.abs(dp))


In [ ]:
dp.shape

In [ ]:
low_freqs = (peak-width / 2) / f_nyquist
high_freqs = (peak+width / 2) / f_nyquist
# Convert input to arrays if they're not already
low_freqs = np.atleast_1d(low_freqs)
high_freqs = np.atleast_1d(high_freqs)

"""Create multiple bandpass filters"""
# Create frequency array
freqs = np.fft.rfftfreq(n)
filt = np.zeros(n)

# Create rectangular bandpass filter
for low, high in zip(low_freqs, high_freqs):
    filt += np.where((abs(freqs) >= low) & (abs(freqs) <= high), 1, 0)

In [ ]:
np.fft.rfftfreq?

